In [2]:
from Bio import SearchIO

/home/shouldsee/.local/lib/python2.7/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


In [21]:
### Manually generated tmp.hmmpf and tmp.domtbl

### 
wkdir = "/data/sequence-data/"
hmm_profile = "%s/tmp.hmmpf" % wkdir
hmm_domtbl = "%s/tmp.domtbl" % wkdir


# kwargs = {"format":"hmmer3-domtab"}
tbl_format = "hmmsearch3-domtab"

In [32]:
parser = SearchIO.parse( hmm_domtbl, tbl_format)

# SearchIO.parse?
# q_hit = next(parser)
i = 0
for q_hits in parser:
#     print q_hit.__dict__
    print q_hits
    i += 1
    if i == 10:
        break
# SearchIO.parse

Program: <unknown program> (<unknown version>)
  Query: cath|4_1_0|16pkA01/5-192 (174)
         <unknown description>
 Target: <unknown target>
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  Q8R965.1/6-383  -
            1      1  P36204.4/6-386  -
            2      1  G0GCY5.1/6-382  -
            3      1  E8N4J2.1/7-379  -
            4      1  A4XKV2.1/9-390  -
            5      1  A0A101H4Q5.1/7-379  -
            6      1  A0A101ECE8.1/6-382  -
            7      1  I7A1P3.1/10-391  -
            8      1  Q67SW6.1/6-380  -
            9      1  L7VTZ3.1/9-386  -
           10      1  H9UJJ7.1/6-381  -
           11      1  F6B4K7.1/6-382  -
           12      1  A0A162N146.1/6-383  -
           13      1  D7CPH0.1/4-381  -
           14      1  F5L4M1.1/6-382  -
           15      1  A0A0F2IYE7.1/8-386  

In [ ]:
# q_hit.to_dict
# SearchIO.to_dict(q_hit)
hit = q_hits[0]
print hit
# hit?
# hit.
hit.evalue
hit.bitscore
hit.id
hit.query_id

In [45]:
### model.py
class hit(object)
# hit.proteindb

Query: cath|4_1_0|16vpA00/47-402
       <unknown description>
  Hit: P06492.1/44-386 (343)
       -
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0  1.1e-136     463.40       ?          [0:303]                [7:310]


'P06492.1/44-386'